# Demand Forecasting with LightGBM & Optuna

This notebook trains a LightGBM model to predict demand, optimized using Optuna.

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import pickle

%matplotlib inline

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load Data
df = pd.read_csv('../data/sales_data.csv')

# Feature Engineering
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Weekday'] = df['Date'].dt.weekday

# Encode Categoricals
cat_cols = ['Store ID', 'Product ID', 'Category', 'Region', 'Weather Condition', 'Seasonality', 'Promotion']
le_dict = {}
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    le_dict[col] = le

# Define Features and Target
# We include Inventory Level as per RPD
features = ['Store ID', 'Product ID', 'Category', 'Region', 'Inventory Level', 
            'Price', 'Discount', 'Weather Condition', 'Promotion', 
            'Competitor Pricing', 'Seasonality', 'Epidemic', 'Month', 'Day', 'Weekday']
target = 'Demand'

X = df[features]
y = df[target]

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Optuna Hyperparameter Tuning

In [3]:
def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 5, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
    }
    
    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return rmse

# Run Optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20) # 20 trials for speed, increase for better results

print('Best trial:', study.best_trial.params)

[I 2025-11-23 02:07:37,721] A new study created in memory with name: no-name-1149bef0-3d13-44b0-91bd-67d573635208
[I 2025-11-23 02:07:46,324] Trial 0 finished with value: 14.290921466764805 and parameters: {'num_leaves': 66, 'learning_rate': 0.02689967673195379, 'n_estimators': 949, 'max_depth': 11, 'min_child_samples': 41, 'subsample': 0.9475826277044406, 'colsample_bytree': 0.8371863798144796}. Best is trial 0 with value: 14.290921466764805.
[I 2025-11-23 02:07:53,531] Trial 1 finished with value: 7.805335048372081 and parameters: {'num_leaves': 67, 'learning_rate': 0.15063139082926058, 'n_estimators': 955, 'max_depth': 7, 'min_child_samples': 69, 'subsample': 0.7046806782247156, 'colsample_bytree': 0.9036450881412936}. Best is trial 1 with value: 7.805335048372081.
[I 2025-11-23 02:07:54,945] Trial 2 finished with value: 14.764999440779485 and parameters: {'num_leaves': 117, 'learning_rate': 0.23034339661310407, 'n_estimators': 153, 'max_depth': 7, 'min_child_samples': 91, 'subsampl

Best trial: {'num_leaves': 67, 'learning_rate': 0.15063139082926058, 'n_estimators': 955, 'max_depth': 7, 'min_child_samples': 69, 'subsample': 0.7046806782247156, 'colsample_bytree': 0.9036450881412936}


## Train Best Model

In [4]:
best_params = study.best_trial.params
best_params['objective'] = 'regression'
best_params['metric'] = 'rmse'

final_model = lgb.LGBMRegressor(**best_params)
final_model.fit(X_train, y_train)

y_pred = final_model.predict(X_test)

print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("R2:", r2_score(y_test, y_pred))

RMSE: 7.805335048372081
MAE: 5.066140436220698
R2: 0.9724143895810476


## Save Model and Encoders

In [5]:
with open('demand_model_lgbm.pkl', 'wb') as f:
    pickle.dump(final_model, f)

with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(le_dict, f)
    
print("Model and encoders saved.")

Model and encoders saved.
